# Imports

In [81]:
import deeplake
import cv2
import os
import xml.etree.ElementTree as ET
import struct
from PIL import Image
import re

In [49]:
ds_downloaded = deeplake.load("hub://activeloop/kuzushiji-kanji")

Opening dataset in read-only mode as you don't have write permissions.


This dataset can be visualized in Jupyter Notebook by ds.visualize() or at https://app.activeloop.ai/activeloop/kuzushiji-kanji

hub://activeloop/kuzushiji-kanji loaded successfully.



In [50]:
type(ds_downloaded)

deeplake.core.dataset.deeplake_cloud_dataset.DeepLakeCloudDataset

In [51]:
ds_downloaded.visualize()

In [52]:
ds_downloaded[0]

Dataset(path='hub://activeloop/kuzushiji-kanji', read_only=True, index=Index([0]), tensors=['images', 'labels'])

In [53]:
img0 = cv2.imread("/home/beav3r/Bachelor_work/Bachelor_work/data/追加/準備ができていない/Unicode/0.png", cv2.IMREAD_GRAYSCALE)
img1 = cv2.imread("/home/beav3r/Bachelor_work/Bachelor_work/data/追加/準備ができていない/Unicode/1.png", cv2.IMREAD_GRAYSCALE)
img_kkanji = cv2.imread("/home/beav3r/Bachelor_work/Bachelor_work/data/kkanji/kkanji2/U+4C61/e6eeb23552e1a21c.png", cv2.IMREAD_GRAYSCALE)
img0 = cv2.resize(img0, (64, 64))
img1 = cv2.resize(img1, (64, 64))
img0 = img0 / 255
img1 = img1 / 255

# Morphology opening and closing
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
img_kkanji_opened = cv2.morphologyEx(img_kkanji, cv2.MORPH_OPEN, kernel)
img_kkanji_closed = cv2.morphologyEx(img_kkanji_opened, cv2.MORPH_CLOSE, kernel)

# Save the thresholded image
cv2.imwrite("/home/beav3r/Bachelor_work/Bachelor_work/data/追加/準備ができていない/Unicode/thresholdedTest.png", img_kkanji_closed)

img_kkanji_closed.shape

(64, 64)

In [54]:
# Count amount of files in subsets and write into txt file

path = "/home/beav3r/Bachelor_work/Bachelor_work/data/kkanji/kkanji2"

def count_files(path):
    for root, dirs, files in os.walk(path):
        for dir in dirs:
            #Save to txt file
            with open("/home/beav3r/Bachelor_work/Bachelor_work/data/kkanji_classes_distribution.txt", "a") as file:
                file.write(dir + " " + str(len(os.listdir(os.path.join(root, dir)))) + "\n")
        
count_files(path)



In [56]:
"""Done"""
# Sort the classes by amount of files

# with open("/home/beav3r/Bachelor_work/Bachelor_work/kkanji_classes_distribution.txt", "r") as file:
#     lines = file.readlines()
#     lines.sort(key=lambda x: int(x.split()[1]), reverse=True)
#     with open("/home/beav3r/Bachelor_work/Bachelor_work/kkanji_classes_distribution_sorted.txt", "w") as file:
#         for line in lines:
#             file.write(line)

'Done'

In [57]:
"""Done"""
# Remote duplicates from the sorted file

# with open("/home/beav3r/Bachelor_work/Bachelor_work/kkanji_classes_distribution_sorted.txt", "r") as file:
#     lines = file.readlines()
#     lines = list(dict.fromkeys(lines))
#     with open("/home/beav3r/Bachelor_work/Bachelor_work/kkanji_classes_distribution_sorted.txt", "w") as file:
#         for line in lines:
#             file.write(line)

'Done'

In [58]:
# Reading svg files
tree = ET.parse("/home/beav3r/Bachelor_work/Bachelor_work/data/kanji_svg/0f9a8.svg")
root = tree.getroot()

for child in root:
    print(child.tag, child.attrib)


{http://www.w3.org/2000/svg}g {'id': 'kvg:StrokePaths_0f9a8', 'style': 'fill:none;stroke:#000000;stroke-width:3;stroke-linecap:round;stroke-linejoin:round;'}
{http://www.w3.org/2000/svg}g {'id': 'kvg:StrokeNumbers_0f9a8', 'style': 'font-size:8;fill:#808080'}


In [76]:
# GNT files exploration
"""Rewrite so it will store images in modules of their names"""

def read_gnt_file(file_name, output_dir):
    try:
        # Ensure output directory exists
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)

        # Open the file in read binary mode
        with open(file_name, 'rb') as images_file:
            # Seek to the end of the file to get its length
            images_file.seek(0, 2)
            file_length = images_file.tell()
            images_file.seek(0)

            n = 0
            while images_file.tell() < file_length:  # Not EOF
                # Read number of bytes for a sample (4 bytes - long)
                char_length = struct.unpack('<l', images_file.read(4))[0]

                # Read GB code (2 bytes)
                char_label = images_file.read(2)

                # Reverse byte order from little-endian to big-endian
                char_label = char_label[::-1]

                # Decode GB2312 into Unicode
                try:
                    char_unicode = char_label.decode('gb2312')
                except UnicodeDecodeError:
                    char_unicode = "unknown" + str(n)

                # Read character width and height (2 bytes each - short)
                char_wid = struct.unpack('<h', images_file.read(2))[0]
                char_hei = struct.unpack('<h', images_file.read(2))[0]

                # Read the bitmap of the grayscale image
                char_image_data = images_file.read(char_wid * char_hei)

                # Convert the raw data into an image using Pillow
                image = Image.frombytes('L', (char_wid, char_hei), char_image_data)

                # Create a valid filename using the label (sanitize the label to avoid invalid characters)
                safe_label = "".join([c if c.isalnum() else "_" for c in char_unicode])
                output_path = os.path.join(output_dir, f"{safe_label}_{n}.png")

                # Save the image as a PNG file
                image.save(output_path)

                n += 1

        print(f"Total images processed: {n}")

    except Exception as e:
        print(f"Error occurred: {e}")

In [110]:
gnt_file = "/home/beav3r/Bachelor_work/Bachelor_work/data/CASIA Online and Offline Chinese Handwriting Databases/Gnt1.0TrainPart1/001-f.gnt"
output_dir = "/home/beav3r/Bachelor_work/Bachelor_work/data/CASIA Online and Offline Chinese Handwriting Databases/My_processed_Gnt1.0TrainPart1"
read_gnt_file(gnt_file, output_dir)

Total images processed: 4024


In [75]:
# Read the first image from processed GNT file

img = cv2.imread("/home/beav3r/Bachelor_work/Bachelor_work/data/CASIA Online and Offline Chinese Handwriting Databases/My_processed_Gnt1.0TrainPart1/__470.png", cv2.IMREAD_GRAYSCALE)
img.shape

(69, 61)

In [125]:
# POT files exploration
"""Old version"""

# def read_pot_file(file_name):
#     try:
#         with open(file_name, 'rb') as pot_file:
#             file_open = True
#             file_length = pot_file.seek(0, 2)  # Seek to end to get length
#             pot_file.seek(0)  # Seek back to beginning

#             n = 0
#             while pot_file.tell() < file_length:  # Not EOF

#                 if file_length - pot_file.tell() < 2:
#                     print("End of file reached1")
#                     break
#                 # Read sample size (2 bytes - unsigned short)
#                 sample_size = struct.unpack('<H', pot_file.read(2))[0]

#                 if file_length - pot_file.tell() < 2:
#                     print("End of file reached2")
#                     break
#                 # Read tag code (4 bytes - DWORD)
#                 tag_code_bytes = pot_file.read(4)

#                 # Reverse byte order and decode GB2312 (Only last two bytes are meaningful)
#                 tag_code = tag_code_bytes[:2][::-1]  # Reverse last two bytes
#                 try:
#                     char_unicode = tag_code.decode('gb2312')
#                 except UnicodeDecodeError:
#                     char_unicode = "unknown" + str(n)

#                 # Sanitize char_unicode to remove non-printable characters
#                 char_unicode = re.sub(r'[^\w]', '_', char_unicode)  # Replace invalid characters

#                 if file_length - pot_file.tell() < 2:
#                     print("End of file reached3")
#                     break
#                 # Read stroke number (2 bytes - unsigned short)
#                 stroke_num = struct.unpack('<H', pot_file.read(2))[0]

#                 print(f"Processing Character: {char_unicode}, Strokes: {stroke_num}, iteration: {n}")

#                 # Read strokes until stroke end (-1, 0) or character end (-1, -1)
#                 strokes = []
#                 for i in range(stroke_num):
#                     stroke = []
#                     while True:
#                         # Read (x, y) coordinates (2 bytes + 2 bytes), sometimes short sometimes signed short
#                         if file_length - pot_file.tell() < 2:
#                             print("End of file reached4")
#                             break
#                         x, y = struct.unpack('<hh', pot_file.read(4))

#                         # Stroke end (-1, 0)
#                         if x == -1 and y == 0:
#                             break
#                         # Character end (-1, -1)
#                         elif x == -1 and y == -1:
#                             break
#                         else:
#                             stroke.append((x, y))

#                     strokes.append(stroke)

#                     # Exit loop if we hit character end
#                     if x == -1 and y == -1:
#                         break

#                 # print(f"Character: {char_unicode}, Strokes: {strokes}")
#                 with open("/home/beav3r/Bachelor_work/Bachelor_work/data/CASIA Online and Offline Chinese Handwriting Databases/My_processed_Pot1.0Train/" + char_unicode + ".txt", "w") as file:
#                     file.write("Strokes: " + str(strokes) + " Stroke number: " + str(stroke_num))
#                 n += 1

#         print(f"Total characters processed: {n}")

#     except Exception as e:
#         print(f"Error occurred: {e}")

In [ ]:
# POT files exploration

def read_pot_file(file_name):
    try:
        with open(file_name, 'rb') as pot_file:
            file_open = True
            file_length = pot_file.seek(0, 2)  # Seek to end to get length
            pot_file.seek(0)  # Seek back to beginning

            n = 0
            while pot_file.tell() < file_length:  # Not EOF

                if file_length - pot_file.tell() < 2:
                    print("End of file reached1")
                    break
                # Read sample size (2 bytes - unsigned short)
                sample_size = struct.unpack('<H', pot_file.read(2))[0]

                if file_length - pot_file.tell() < 2:
                    print("End of file reached2")
                    break
                # Read tag code (4 bytes - DWORD)
                tag_code_bytes = pot_file.read(4)

                # Reverse byte order and decode GB2312 (Only last two bytes are meaningful)
                tag_code = tag_code_bytes[:2][::-1]  # Reverse last two bytes
                try:
                    char_unicode = tag_code.decode('gb2312')
                except UnicodeDecodeError:
                    char_unicode = "unknown" + str(n)

                # Sanitize char_unicode to remove non-printable characters
                char_unicode = re.sub(r'[^\w]', '_', char_unicode)  # Replace invalid characters

                if file_length - pot_file.tell() < 2:
                    print("End of file reached3")
                    break
                # Read stroke number (2 bytes - unsigned short)
                stroke_num = struct.unpack('<H', pot_file.read(2))[0]

                print(f"Processing Character: {char_unicode}, Strokes: {stroke_num}, iteration: {n}")

                # Read strokes until stroke end (-1, 0) or character end (-1, -1)
                strokes = []
                for i in range(stroke_num):
                    stroke = []
                    while True:
                        # Read (x, y) coordinates (2 bytes + 2 bytes), sometimes short sometimes signed short
                        if file_length - pot_file.tell() < 2:
                            print("End of file reached4")
                            break
                        x, y = struct.unpack('<hh', pot_file.read(4))

                        # Stroke end (-1, 0)
                        if x == -1 and y == 0:
                            break
                        # Character end (-1, -1)
                        elif x == -1 and y == -1:
                            break
                        else:
                            stroke.append((x, y))

                    strokes.append(stroke)

                    # Exit loop if we hit character end
                    if x == -1 and y == -1:
                        break

                # print(f"Character: {char_unicode}, Strokes: {strokes}")
                with open("/home/beav3r/Bachelor_work/Bachelor_work/data/CASIA Online and Offline Chinese Handwriting Databases/My_processed_Pot1.0Train/" + char_unicode + ".txt", "w") as file:
                    file.write("Strokes: " + str(strokes) + " Stroke number: " + str(stroke_num))
                n += 1

        print(f"Total characters processed: {n}")

    except Exception as e:
        print(f"Error occurred: {e}")

In [126]:
pot_file = "/home/beav3r/Bachelor_work/Bachelor_work/data/CASIA Online and Offline Chinese Handwriting Databases/Pot1.0Train/001.pot"
read_pot_file(pot_file)

Processing Character: __, Strokes: 2, iteration: 0
Processing Character: unknown1, Strokes: 8704, iteration: 1
Processing Character: __, Strokes: 4, iteration: 2
Processing Character: unknown3, Strokes: 9216, iteration: 3
Processing Character: __, Strokes: 2, iteration: 4
Processing Character: unknown5, Strokes: 9728, iteration: 5
Processing Character: __, Strokes: 1, iteration: 6
Processing Character: unknown7, Strokes: 10496, iteration: 7
Processing Character: __, Strokes: 3, iteration: 8
Processing Character: unknown9, Strokes: 11008, iteration: 9
Processing Character: __, Strokes: 1, iteration: 10
Processing Character: unknown11, Strokes: 11520, iteration: 11
Processing Character: __, Strokes: 1, iteration: 12
Processing Character: unknown13, Strokes: 12032, iteration: 13
Processing Character: 0_, Strokes: 1, iteration: 14
Processing Character: unknown15, Strokes: 12544, iteration: 15
Processing Character: 2_, Strokes: 1, iteration: 16
Processing Character: unknown17, Strokes: 1305